In [2]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [3]:
df = pd.read_csv('C:\\Users\\SSAFY\\Desktop\\ssafy\\DA_pjt\\S06P22E202\\data\\DataCrawling\\정재호\\musinsa_clothes_top.csv')

1. 일단 전부 돌려보자
2. 만약 결과값이 안 좋으면 임의적으로 category랑 color를 줄여야한다...

1. 아니면 추천알고리즘을 돌릴 데이터를 각 largeCategory별로 나눠서 총 5번의 추천 알고리즘을 돌려라!!!

1. user가 봄을 좋아하면(1) item을 추천할 때, 그냥 봄 위주로 추천
2. user가 봄을 좋아하면(1) item(4개의 변수), 학습시킬 때, rable data(거래목록) 기준으로 학습 시 자동으로 될 것이다!
3. user data와 item data 모두 원핫인코딩을 해줘야한다!!

0. knn & mf & mf+plsi & lightFM
1. 리뷰 평점을 7:2:1로 train:validation:test로 나눠서 정확도 얻기
2. 0의 비율보다 1의 비율이 엄청나게 작을 것이다. => accuracy로만 측정하기에는 0로 예측하는 모델이 99.9% 나올 것이다.(3번을 쓰는 이유)
3. roc&auc / f1 score(정확도) => 2번의 문제점을 최대한 보완하며 기준을 정할 수 있다. (0도 맞추면서, 최대한 1의 값이 맞는지로 비교)

In [15]:
df.head(100)

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,...,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
0,1907183,1,상의,106,니트/스웨터,56,우먼즈 니티드 폴로 셔츠 [테라코타],무신사 스탠다드,//image.msscdn.net/images/goods_img/20210419/1...,"43,900원",...,191,2,2,5,3,2,0,1,0,0
1,2104168,1,상의,106,니트/스웨터,56,다이드 워셔블 니트 풀오버 [CORAL BROWN],블러프,//image.msscdn.net/images/goods_img/20210902/2...,"42,000원",...,0,0,0,0,0,0,0,0,0,0
2,1864012,1,상의,110,긴팔 티셔츠,56,헤비웨이트 릴렉스드 크루 넥 긴팔 티셔츠 [레드 빈],무신사 스탠다드,//image.msscdn.net/images/goods_img/20210325/1...,"19,900원",...,"2,681",4,3,4,5,4,1,0,1,0
3,1212027,1,상의,106,니트/스웨터,56,617 ramsoul round knit brick,에이본,//image.msscdn.net/images/goods_img/20191104/1...,"48,000원",...,5,0,0,0,0,0,0,0,0,0
4,2228389,1,상의,106,니트/스웨터,56,M12 raglan over knit wine,에이본,//image.msscdn.net/images/goods_img/20211111/2...,"56,000원",...,0,3,1,5,5,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2113755,1,상의,106,니트/스웨터,40,BOLERO CROP KNIT SET CHARCOAL,어반드레스,//image.msscdn.net/images/goods_img/20210907/2...,"27,860원",...,165,0,0,0,0,0,0,0,0,0
96,2113751,1,상의,106,니트/스웨터,40,BOLERO CROP KNIT SET DARKVIOLET,어반드레스,//image.msscdn.net/images/goods_img/20210907/2...,"27,860원",...,165,3,1,5,5,4,1,0,1,1
97,1608569,1,상의,104,후드 티셔츠,40,Y.E.S PATH HOODIE Violet,예스아이씨,//image.msscdn.net/images/goods_img/20200917/1...,"36,800원",...,310,0,0,0,0,0,0,0,0,0
98,1778793,1,상의,104,후드 티셔츠,40,Unisex 엠보 로고 후디 바이올렛 (BX1241S31S),브룩스,//image.msscdn.net/images/goods_img/20210204/1...,"39,500원",...,1,0,0,0,0,0,0,0,0,0


In [4]:
print(df.groupby('colorNo').count())

         clothId  largeCategory  largeCategoryName  smallCategory  \
colorNo                                                             
4            475            475                475            475   
5            489            489                489            489   
8            476            476                476            476   
15          5242           5242               5242           5242   
16           231            231                231            231   
17           204            204                204            204   
18           883            883                883            883   
19          1881           1881               1881           1881   
20            32             32                 32             32   
21            77             77                 77             77   
22           463            463                463            463   
26           316            316                316            316   
27           232            232   

print(df.shape)

In [6]:
print(len(list(df['clothId'].unique())))

15910


In [27]:
a = ('a', 'b', 'c', 'd', 'e')
a = list(a)
print(a[0:5])

['a', 'b', 'c', 'd', 'e']


In [28]:
print(482*90)

43380
